# Categorization using log-likelihood of word2vec models

In [15]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from pandas import DataFrame
import numpy as np

construct the file names for the learned models and the validation sets

In [16]:
category_names = ['Sonstiges', 'Lifestyle', 
          'Wirtschaft', 'Finanzen', 'Lokal', 
          'Politik', 'Sport', 'Technologie', 'Kultur']

num_models = len(category_names)
model_paths = ["data/corpus{}+base.word2vec.model".format(x) for x in category_names]
validation_paths = ["data/corpus{}.validation.txt".format(x) for x in category_names]

load the validation sets

In [17]:
num_validation_entries = [sum([1 for _ in x]) for x in validators]
validation_stats = DataFrame(num_validation_entries, category_names, ['dim(V_i)'])
print('number of articles used for validation:')
print(validation_stats) 

number of articles used for validation:
             dim(V_i)
Sonstiges         770
Lifestyle         328
Wirtschaft        817
Finanzen          333
Lokal             249
Politik          1596
Sport             449
Technologie       361
Kultur            217


## Validation

validate each validation set with all models and calculate the score (log likelihood) of each sentence for each model.

scores is an array in the form:

    [for each model:
        [for each validation set:
            [score of each sentence]
        ]
    ]

**Note**:
There is no cross validation implemented, since it would require learning the category models ${ M }_{ i }$ on-the-fly which is in the current implementation too needy for memory

In [18]:
def get_scores(model_path):
    # load the model
    print('loading model {}'.format(model_path))
    model =  Word2Vec.load(model_path)
    # calculate the score (log likelihood) of each validation set for this model
    print('validating model...')
    scores = [model.score(validator) for validator in validators]
    return scores

main loop that calculates the scores for all models

In [19]:
#container to hold the calculated scores
scores = []

for model in model_paths:
    model_scores = get_scores(model)
    scores.append(model_scores)

loading model data/corpusSonstiges+base.word2vec.model
validating model...
loading model data/corpusLifestyle+base.word2vec.model
validating model...
loading model data/corpusWirtschaft+base.word2vec.model
validating model...
loading model data/corpusFinanzen+base.word2vec.model
validating model...
loading model data/corpusLokal+base.word2vec.model
validating model...
loading model data/corpusPolitik+base.word2vec.model
validating model...
loading model data/corpusSport+base.word2vec.model
validating model...
loading model data/corpusTechnologie+base.word2vec.model
validating model...
loading model data/corpusKultur+base.word2vec.model
validating model...


## Log Likelihood

output the average score (log likelihood) of all sentences in a validation set for one model

row = average likelihood that an item of this category is generated by the model in the row

e.g: the lowest value in each column is the category a sentence of this model is most likely classified to

$$ { S }_{ i,j }=\frac { \sum _{ s=1 }^{ dim({ V }_{ i }) }{ score({ S(V }_{ i },\quad s),\quad { M }_{ j }) }  }{ dim({ V }_{ i }) } $$

where:
* ${ V }_{ i } $ is the validationset for category $i$
* ${ M }_{ i } $ is the model trained for category $i$
* ${ S }_{ i,j }$ is the score ($i$ is the column, $j$ is the row in the table)
* $S({V}_{i}, x)$ is the $x$th sentence in the validation set ${V}_{i}$
* $dim({ V }_{ i })$ is the number of elements in the validation set
* $score(s, m)$ is the log-likelihood that the word is generated by the model [[Taddy, Matt. Document Classification by Inversion of Distributed Language Representations](https://arxiv.org/pdf/1504.07295.pdf)]


In [20]:
average_scores = []
for score_set in scores:
    average_scores.append([sum(x) / len(x) for x in score_set])

#transpose the array before creating the DataFrame because pandas is row-oriented
result = DataFrame(np.transpose(average_scores), category_names, category_names)
print(result)

               Sonstiges    Lifestyle   Wirtschaft     Finanzen        Lokal  \
Sonstiges   -3023.305563 -3316.458243 -3458.459421 -3502.693819 -3505.015840   
Lifestyle   -3470.167490 -3230.019704 -3379.121821 -3694.238708 -3715.646034   
Wirtschaft  -2255.291836 -2080.033885 -1850.201417 -2273.048428 -2399.692025   
Finanzen    -2819.153567 -3010.546884 -2862.410102 -2414.655164 -3275.942700   
Lokal       -5072.434925 -5172.428481 -5601.607124 -5249.026092 -3597.520388   
Politik     -3259.656002 -3270.765876 -3251.658240 -3473.375445 -3537.727668   
Sport       -4126.546658 -4262.982045 -4460.560663 -4381.310551 -4225.625471   
Technologie -3697.492521 -3691.743123 -3755.177561 -3831.746594 -3912.820028   
Kultur      -9062.581682 -9165.819357 -9754.343478 -9762.327799 -9793.236392   

                 Politik        Sport  Technologie       Kultur  
Sonstiges   -3379.924037 -3522.187860 -3398.669042 -3492.243594  
Lifestyle   -3449.871508 -3675.448228 -3533.370444 -3649.240762  
W

## Classification Quality
calculate the number of categorizations for every category

rows = categories of training set

columns = number of items of the train set classified in the category

e.g: the highest number in each row should be on the diagonal of the matrix

first step is to transpose the model (switch the first two dimensions from model->category to category->model), then numpy.argmax is used to find the index of the model that has the highest score for this category

$${ C }_{ i,j }=\sum { \begin{cases} 1\quad if\quad { S }_{ i,j }\quad >\quad \underset { k\in { V }\setminus { V }_{ i } }{ max({ S }_{ k,j }) }  \\ 0\quad otherwise \end{cases} } $$

where:
* ${ C }_{ i,j }$ is the number of elements in ${V}_{i}$ that have are classified to ${M}_{i}$
* ${ S }_{ i,j }\quad >\quad \underset { k\in { V }\setminus { V }_{ i } }{ max({ S }_{ k,j }) } $ is the classification rule that assigns the article to the class that yields the highest score for the model

In [21]:
classification_matrix = np.empty([num_models, num_models], dtype=int)

for category_index in range(num_models):
    #transpose the scores array to form [model][category][sentence_score] to [category][model][sentence_score]
    category_scores = [model[category_index] for model in scores]
    #get the classification matrix in each model
    #the values represent the category index they were assigned to
    classifications = np.argmax(category_scores, axis = 0)
    
    #convert the classification matrix to a count of classification in each category
    classification_count = [np.sum(classifications == x) for x in range(len(category_names))]
    classification_matrix[category_index]=classification_count
    
result = DataFrame(classification_matrix, category_names, category_names)
print(result)   

             Sonstiges  Lifestyle  Wirtschaft  Finanzen  Lokal  Politik  \
Sonstiges          452        154          40        10     10       58   
Lifestyle           52        182          29         6      0       29   
Wirtschaft          29         52         593        67      2       30   
Finanzen            10          9          98       208      0        3   
Lokal               21         12          21         1    158       30   
Politik            115         46         121        13      7     1280   
Sport               13          3          10         2      1        0   
Technologie         14         33          29         3      0       10   
Kultur              46         62           8         2      2        2   

             Sport  Technologie  Kultur  
Sonstiges        5           19      22  
Lifestyle        4           16      10  
Wirtschaft       1           41       2  
Finanzen         0            5       0  
Lokal            0            5       1

## Accuracy

Calculate the accuracy matrix. Accuracy is defined as 
$\frac { TP +TN }{ total\ elements }$

$${ ACC }_{ i,j }=\frac { { C }_{ i,j } }{ \sum { { C }_{ j } }  } $$

**Note**:
True accuracy measures as per definition are only found on the diagonal of the matrix. The other values are the ratio of false positives in the corresponding category

In [22]:
# the max(, 1) function surrounding sum makes sure wo don't divide by 0 if no match occurred
accuracy_matrix = [category / float(max([sum(category) ,1])) for category in classification_matrix]

result = DataFrame(accuracy_matrix, category_names, category_names)
print(result) 

             Sonstiges  Lifestyle  Wirtschaft  Finanzen     Lokal   Politik  \
Sonstiges     0.587013   0.200000    0.051948  0.012987  0.012987  0.075325   
Lifestyle     0.158537   0.554878    0.088415  0.018293  0.000000  0.088415   
Wirtschaft    0.035496   0.063647    0.725826  0.082007  0.002448  0.036720   
Finanzen      0.030030   0.027027    0.294294  0.624625  0.000000  0.009009   
Lokal         0.084337   0.048193    0.084337  0.004016  0.634538  0.120482   
Politik       0.072055   0.028822    0.075815  0.008145  0.004386  0.802005   
Sport         0.028953   0.006682    0.022272  0.004454  0.002227  0.000000   
Technologie   0.038781   0.091413    0.080332  0.008310  0.000000  0.027701   
Kultur        0.211982   0.285714    0.036866  0.009217  0.009217  0.009217   

                Sport  Technologie    Kultur  
Sonstiges    0.006494     0.024675  0.028571  
Lifestyle    0.012195     0.048780  0.030488  
Wirtschaft   0.001224     0.050184  0.002448  
Finanzen     0.000000

calculate the average accuracy of the diagonal to get a accuracy score

$$score=\frac { \sum _{ i=0 }^{ dim({ V }_{ i }) }{ { ACC }_{ i,i } * dim({ V }_{ i }) }  }{  \sum _{ i=0 }^{ dim({ V }_{ i }) }{dim({ V }_{ i }) }   } $$

This value is comparable with the accuracy validation score of scikitlearn

In [23]:
true_positives = 0.0
num_samples = 0
for x in range(num_models):
    true_positives += classification_matrix[x][x]
    num_samples += sum(classification_matrix[x])
    
average_score = true_positives / num_samples

print('score: {}'.format(average_score))

score: 0.7091796875
